In [1]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SQLContext
sc = SparkContext()
sqlContext = SQLContext(sc)

In [4]:
company_df = sqlContext.read.format("com.databricks.spark.csv").options(header="true", inferschema="true").load("C:\\Users\\12499\\Downloads\\COVID19 cases.csv")
company_df.take(1)

[Row(_id=15339, Assigned_ID=1, Outbreak Associated='Sporadic', Age Group='50 to 59 Years', Neighbourhood Name='Willowdale East', FSA='M2N', Source of Infection='Travel', Classification='CONFIRMED', Episode Date='2020-01-22', Reported Date='2020-01-23', Client Gender='FEMALE', Outcome='RESOLVED', Currently Hospitalized='No', Currently in ICU='No', Currently Intubated='No', Ever Hospitalized='No', Ever in ICU='No', Ever Intubated='No')]

In [5]:
company_df.cache()
company_df.printSchema()

root
 |-- _id: integer (nullable = true)
 |-- Assigned_ID: integer (nullable = true)
 |-- Outbreak Associated: string (nullable = true)
 |-- Age Group: string (nullable = true)
 |-- Neighbourhood Name: string (nullable = true)
 |-- FSA: string (nullable = true)
 |-- Source of Infection: string (nullable = true)
 |-- Classification: string (nullable = true)
 |-- Episode Date: string (nullable = true)
 |-- Reported Date: string (nullable = true)
 |-- Client Gender: string (nullable = true)
 |-- Outcome: string (nullable = true)
 |-- Currently Hospitalized: string (nullable = true)
 |-- Currently in ICU: string (nullable = true)
 |-- Currently Intubated: string (nullable = true)
 |-- Ever Hospitalized: string (nullable = true)
 |-- Ever in ICU: string (nullable = true)
 |-- Ever Intubated: string (nullable = true)



In [7]:
company_df.describe().toPandas().transpose()

,0,1,2,3,4
summary,count,mean,stddev,min,max
_id,15432,23054.5,4454.979012296242,15339,30770
Assigned_ID,15432,7727.875194401244,4468.050671189019,1,15485
Outbreak Associated,15432,None,None,Outbreak Associated,Sporadic
Age Group,15406,None,None,19 and younger,90 and older
Neighbourhood Name,14826,None,None,Agincourt North,Yorkdale-Glen Park
FSA,14872,None,None,M1B,M9W
Source of Infection,15432,None,None,Close contact,Unknown/Missing
Classification,15432,None,None,CONFIRMED,PROBABLE
Episode Date,15432,None,None,2020-01-21,2020-08-03


In [18]:
import six
for i in company_df.columns:
    if not( isinstance(company_df.select(i).take(1)[0][0], six.string_types)):
        print( "Correlation to id for ", i, company_df.stat.corr('_id',i))

Correlation to id for  _id 1.0
Correlation to id for  Assigned_ID 0.9999987139408719


In [20]:
from pyspark.ml.feature import VectorAssembler
vectorAssembler = VectorAssembler(inputCols = ['_id', 'Assigned_ID'], outputCol = 'features')
tcompany_df = vectorAssembler.transform(company_df)
tcompany_df = tcompany_df.select(['features', '_id'])
tcompany_df.show(3)

+-------------+-----+
|     features|  _id|
+-------------+-----+
|[15339.0,1.0]|15339|
|[15340.0,2.0]|15340|
|[15341.0,3.0]|15341|
+-------------+-----+
only showing top 3 rows



In [21]:
splits = tcompany_df.randomSplit([0.7, 0.3])
train_df = splits[0]
test_df = splits[1]

In [23]:
from pyspark.ml.regression import LinearRegression
lr = LinearRegression(featuresCol = 'features', labelCol='_id', maxIter=10, regParam=0.3, elasticNetParam=0.8)
lr_model = lr.fit(train_df)
print("Coefficients: " + str(lr_model.coefficients))
print("Intercept: " + str(lr_model.intercept))

Coefficients: [0.5443612188139115,0.4542319644585528]
Intercept: 6994.258998797904
